In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import os
import cv2
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import json
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input
from tensorflow.keras.layers import Activation,GlobalAveragePooling2D, Dense, BatchNormalization, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import SGD


In [ ]:
# Load the InceptionV3 model without the top layer (include_top=False)
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Add new layers to the model for our classification task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(5, activation='softmax')(x)

# This is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)


87910968/87910968 [==============================] - 3s 0us/step


In [ ]:
# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Agritariat/Data/train.csv")
df

,image_id,label
0,1000015157.jpg,0
1,1000201771.jpg,3
2,100042118.jpg,1
3,1000723321.jpg,1
4,1000812911.jpg,3
...,...,...
21392,999068805.jpg,3
21393,999329392.jpg,3
21394,999474432.jpg,1
21395,999616605.jpg,4


In [ ]:
with open("/content/drive/MyDrive/Agritariat/Data/label_num_to_disease_map.json", "r") as file:
    label_name = json.load(file)
label_name

{'0': 'Cassava Bacterial Blight (CBB)',
 '1': 'Cassava Brown Streak Disease (CBSD)',
 '2': 'Cassava Green Mottle (CGM)',
 '3': 'Cassava Mosaic Disease (CMD)',
 '4': 'Healthy'}

In [ ]:
image_path = "/content/drive/MyDrive/Agritariat/Data/train_images"
image_list = os.listdir(image_path)

In [ ]:
# Matching labels with images
df = df[df["image_id"].isin(image_list)]
df.reset_index(drop=True, inplace=True)

In [ ]:
# create paths of the image and label
path_name = "/content/drive/MyDrive/Agritariat/Data/train_images/"
df["image_path"] = df["image_id"].apply(lambda x: str(path_name+x))
df["label_name"] = df["label"].apply(lambda x: label_name.get(str(x), "Unknown"))
df.head()


<ipython-input-15-0acebb643273>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["image_path"] = df["image_id"].apply(lambda x: str(path_name+x))
<ipython-input-15-0acebb643273>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label_name"] = df["label"].apply(lambda x: label_name.get(str(x), "Unknown"))


,image_id,label,image_path,label_name
0,157078263.jpg,3,/content/drive/MyDrive/Agritariat/Data/train_i...,Cassava Mosaic Disease (CMD)
1,1574893536.jpg,3,/content/drive/MyDrive/Agritariat/Data/train_i...,Cassava Mosaic Disease (CMD)
2,1575013487.jpg,3,/content/drive/MyDrive/Agritariat/Data/train_i...,Cassava Mosaic Disease (CMD)
3,1576606254.jpg,0,/content/drive/MyDrive/Agritariat/Data/train_i...,Cassava Bacterial Blight (CBB)
4,1579761476.jpg,1,/content/drive/MyDrive/Agritariat/Data/train_i...,Cassava Brown Streak Disease (CBSD)


In [ ]:
epochs = 20
batch_size = 32
image_height = 64
image_width = 64

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_df = df.sample(frac=0.8, random_state=42) # Splitting 80% for training
test_df = df.drop(train_df.index) # Use remaining 20% for testing
# Define ImageDataGenerator for training and testing
train_datagen = ImageDataGenerator(
    rescale=1.0/255,  # Rescale pixel values to [0, 1]
    rotation_range=20,  # Randomly rotate images in the range 0-20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by 20% of the width
    height_shift_range=0.2,  # Randomly shift images vertically by 20% of the height
    shear_range=0.2,  # Shear intensity (shear angle in counter-clockwise direction as radians)
    zoom_range=0.2,  # Randomly zoom in or out on images
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Points outside the boundaries are filled according to the nearest pixel
)

test_datagen = ImageDataGenerator(
    rescale=1.0/255  # Only rescale pixel values to [0, 1]
)

# Use the data generators with flow_from_dataframe
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    x_col="image_path",
    y_col="label_name",
    target_size=(image_height, image_width),
    batch_size=batch_size,
    color_mode="rgb",
    class_mode="categorical"
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col="image_path",
    y_col="label_name",
    target_size=(image_height, image_width),
    batch_size=batch_size,
    color_mode="rgb",
    class_mode="categorical"
)

# Get sample numbers for fit and validation
nb_train_samples = train_generator.samples
nb_test_samples = test_generator.samples
classes = list(train_generator.class_indices.keys())


Found 14270 validated image filenames belonging to 5 classes.
Found 3568 validated image filenames belonging to 5 classes.


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=test_generator, # Changed validation_generator to test_generator
    validation_steps=test_generator.samples // test_generator.batch_size, # Changed validation_generator to test_generator
    epochs=20
)

Epoch 1/20
310/445 [===================>..........] - ETA: 38:48 - loss: nan - accuracy: 0.0521

In [ ]:
#Unfreeze some layers for fine-tuning
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Re-compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continue training (fine-tuning)
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20
)


ValueError: Must provide at least one structure

In [ ]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {val_accuracy * 100:.2f}%')

# Plot the training and validation accuracy and loss
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper left')
plt.show()


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(None, 1), output.shape=(None, 5)